In [4]:
#!pip install openai langchain langchain-community chromadb langchain[pdf] tiktoken python-dotenv pypdf streamlit

In [5]:
import os
from dotenv import load_dotenv

load_dotenv()
openai_api_key = os.getenv("OPENAI_API_KEY")

In [7]:
from langchain.vectorstores import Chroma
from langchain.chat_models import ChatOpenAI
from langchain.embeddings import OpenAIEmbeddings
from langchain.schema.output_parser import StrOutputParser
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.schema.runnable import RunnablePassthrough
from langchain.prompts import PromptTemplate
from langchain.vectorstores.utils import filter_complex_metadata

def initialize_model():
    """Initialize the OpenAI chat model."""
    return ChatOpenAI(model="gpt-4", temperature=0)

def initialize_prompt():
    """Create and return a prompt template."""
    return PromptTemplate.from_template(
        """
        <s> [INST] You are an assistant for question-answering tasks. Use the following pieces of retrieved context 
        to answer the question. If you don't know the answer, just say that you don't know. Use three sentences
        maximum and keep the answer concise. [/INST] </s> 
        [INST] Question: {question} 
        Context: {context} 
        Answer: [/INST]
        """
    )

def ingest_pdf(pdf_file_path: str):
    """Ingest a PDF, process it into chunks, and create a retriever."""
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1024, chunk_overlap=100)
    docs = PyPDFLoader(file_path=pdf_file_path).load()
    chunks = text_splitter.split_documents(docs)
    chunks = filter_complex_metadata(chunks)

    vector_store = Chroma.from_documents(documents=chunks, embedding=OpenAIEmbeddings())
    retriever = vector_store.as_retriever(
        search_type="similarity_score_threshold",
        search_kwargs={
            "k": 3,
            "score_threshold": 0.5,
        },
    )
    return retriever

def build_chain(retriever, model, prompt):
    """Build the question-answering chain."""
    return ({"context": retriever, "question": RunnablePassthrough()}
            | prompt
            | model
            | StrOutputParser())

def ask_question(chain, query: str):
    """Ask a question using the chain."""
    if not chain:
        return "Please, add a PDF document first."

    return chain.invoke(query)

def clear_chain():
    """Clear all components for resetting the system."""
    return None, None, None

In [ ]:
# Example usage

if __name__ == "__main__":
    # Initialize components
    model = initialize_model()
    prompt = initialize_prompt()

    # Ingest PDF
    pdf_file_path = "example.pdf"  # Replace with the actual PDF file path
    retriever = ingest_pdf(pdf_file_path)

    # Build the chain
    chain = build_chain(retriever, model, prompt)

    # Ask a question
    query = "What is the summary of the document?"
    response = ask_question(chain, query)
    print(response)

    # Clear the system
    retriever, chain, model = clear_chain()


2024-12-11 13:52:50.034 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-12-11 13:52:50.038 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-12-11 13:52:50.039 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-12-11 13:52:50.041 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-12-11 13:52:50.042 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-12-11 13:52:50.042 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-12-11 13:52:50.042 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-12-11 13:52:50.044 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar

AttributeError: module 'streamlit' has no attribute 'run'

Usage: streamlit run [OPTIONS] TARGET [ARGS]...
Try 'streamlit run --help' for help.

Error: Missing argument 'TARGET'.
